# Word2Vec實作
- 字詞所代表的意義非常多元，在不同狀況下，會代表不同意思。要把多元意思用單一向量表示，則必須要進行word embedding的動作，也就是把高維向量降為低維向量的過程
- 之前介紹過，利用分散式表示法來表達字詞向量，例如PMI、SVD..統計法..等
- 2013年神經網路盛行後，Tomas Mikolov利用神經網路訓練方式，來獲得字詞的表達向量，獲得很棒的成果。一般認為是利用神經網路模擬人類的理解能力，獲得不錯的分布空間所得到的成果。
- 本範例以維基百科wiki部分資料作範例
- 資料來源：https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2
- 利用結巴分詞(jieba)進行斷詞，gensim套件進行word2vec計算
- 本範例約需1小時長時間執行


In [ ]:
!wget https://dumps.wikimedia.org/zhwiki/20231201/zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2

### opencc是繁簡轉換工具

In [ ]:
!pip install opencc-python-reimplemented

### gensim是訓練word2vec的函式庫

In [ ]:
from gensim.corpora import WikiCorpus

wiki_corpus = WikiCorpus('zhwiki-20231201-pages-articles-multistream1.xml-p1p187712.bz2', dictionary={})

In [ ]:
wiki_corpus

In [ ]:
next(iter(wiki_corpus.get_texts()))[:10]

## 把wiki的資料檔案，轉換成連續文字的txt檔案

In [ ]:
text_num = 0

with open('wiki_text.txt', 'w', encoding='utf-8') as f:#打開一個名為 'wiki_text.txt' 的文本文件
    for text in wiki_corpus.get_texts():#迭代 wiki_corpus 中的文本數據
        f.write(' '.join(text)+'\n')# 將每個文本數據轉換為以空格分隔的字詞（使用 join 方法），然後將其寫入文件。每個文本之間以換行符 \n 分隔
        text_num += 1#計數處理的文本數目。
        if text_num % 10000 == 0:#每處理 10000 篇文本，輸出一條消息，顯示已處理的文章數
            print('{} articles processed.'.format(text_num))#在程序結束時，打印總處理的文章數

    print('{} articles processed.'.format(text_num))

In [ ]:
import jieba
from opencc import OpenCC# 創建一個 OpenCC（開放中文轉換）的實例


# Initial
cc = OpenCC('s2t')
train_data = open('wiki_text.txt', 'r', encoding='utf-8').read()# 打開 'wiki_text.txt' 文件，讀取其中的文本數據
train_data = cc.convert(train_data)# 將讀取的文本數據進行簡體到繁體的轉換
train_data = jieba.lcut(train_data)#使用 jieba 分詞工具對文本進行分詞，得到一個詞彙列表
train_data = [word for word in train_data if word != '']#過濾掉空的詞彙
train_data = ' '.join(train_data)#將詞彙列表組合成一個以空格分隔的字串
open('seg.txt', 'w', encoding='utf-8').write(train_data)#打開 'seg.txt' 文件，將處理後的文本寫入文件

In [ ]:
from gensim.models import word2vec


# Settings
seed = 666
sg = 0
window_size = 10
#vector_size = 100
min_count = 1
workers = 8
#epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('seg.txt')#讀取 'seg.txt' 文件中的分詞文本數據。LineSentence 是 gensim 提供的一個類別，用於從文本文件中讀取每行作為一個文檔。
model = word2vec.Word2Vec(#創建 Word2Vec 模型。其中的參數包括
    train_data,
    min_count=min_count,#詞彙表中的詞的最小出現次數，出現次數少於此值的詞將被忽略
    #size=vector_size,
    workers=workers,#訓練模型時使用的並行工作數
    #iter=epochs,
    window=window_size,#考慮每個目標詞的上下文詞的最大距離。
    sg=sg,
    seed=seed,#隨機數生成的種子，確保每次訓練的結果一致
    batch_words=batch_words# 每個工作進程處理的單詞數目
)

model.save('word2vec.model')

In [ ]:
from gensim.models import word2vec#載入之前訓練好的 Word2Vec 模型，該模型儲存在 'word2vec.model' 文件中

string = '電腦'
model = word2vec.Word2Vec.load('word2vec.model')
print(string)

for item in model.wv.most_similar(string):
    print(item)